In [1]:
from PIL import Image
from io import BytesIO
from transformers import CLIPProcessor, CLIPModel, SegformerImageProcessor, AutoModelForSemanticSegmentation , AutoFeatureExtractor
from urllib.parse import urlparse, urlunparse
from weaviate.util import generate_uuid5
import torch
import torch.nn as nn
import numpy as np
import pandas as pd
import os
import weaviate
import numpy as np
from numpy.linalg import norm
import json
import cv2
import base64
from time import sleep
import ast
import re

c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
c:\Users\Jayesh\AppData\Local\Programs\Python\Python39\lib\site-packages\bitsandbytes\cextension.py:34: UserWarning: The installed version of bitsandbytes was compiled without GPU support. 8-bit optimizers, 8-bit multiplication, and GPU quantization are unavailable.
  warn("The installed version of bitsandbytes was compiled without GPU support. "


'NoneType' object has no attribute 'cadam32bit_grad_fp32'


In [2]:
client = weaviate.Client(url="http://localhost:8080")

In [3]:
customer = open("customer.json", "r")
customer_class = json.load(customer)
customer.close()
client.schema.delete_class("Customer_Profile")
client.schema.create_class(customer_class)

In [4]:
checkpoint = "patrickjohncyh/fashion-clip"
model = CLIPModel.from_pretrained(checkpoint)
processor = CLIPProcessor.from_pretrained(checkpoint)

def getTextEmbeddings(text):
	inputs = processor(text=text , images=Image.new('RGB' , (72 , 72)), return_tensors="pt", padding=True)
	outputs = model(**inputs , return_dict=True)
	return outputs["text_embeds"]

In [ ]:
PATH = "D:\\Codes\\ziggy-ai\\scraping\\Flipkart"

In [ ]:
df1 = pd.read_csv(f"{PATH}\\Girls Tops 1.csv")
df2 = pd.read_csv(f"{PATH}\\Girls Tops 2.csv")
df3 = pd.read_csv(f"{PATH}\\Girls Bottoms.csv")
df4 = pd.read_csv(f"{PATH}\\Formal Shirts.csv")
df5 = pd.read_csv(f"{PATH}\\Formal Pants.csv")

In [ ]:
user_profile1_top = []
user_profile2_top = []
user_profile3_top = []
user_profile4_top = []
user_profile1_bottom = []
user_profile2_bottom = []
user_profile3_bottom = []
user_profile4_bottom = []

In [ ]:
def clean_url(url):
    parsed_url = urlparse(url)
    cleaned_parsed_url = parsed_url._replace(query='')
    cleaned_url = urlunparse(cleaned_parsed_url)
    return cleaned_url

def extract_first_amount(text):
    amount_match = re.search(r'₹(\d+)', text)
    if amount_match:
        first_amount = amount_match.group(1)
        return int(first_amount)
    else:
        return None

def get_rating_and_number(string1, string2):
    rating = string1[0:(len(string1)-len(string2))]
    ratings_match = re.search(r'([\d,.]+) ratings', string2)
    if ratings_match:
        num_ratings = int(ratings_match.group(1).replace(',', ''))
    else:
        num_ratings = 0
    return rating, num_ratings

def get_product_details(dict_string):
    dict = ast.literal_eval(dict_string)
    keys_to_check = ['Color', 'Pattern', 'Fabric', 'Fit', 'Type']
    new_dict = {key: dict.get(key, None) for key in keys_to_check}
    return new_dict

In [ ]:
def mean_pooling(vector_list):
    vectors_array = np.array(vector_list)
    mean_pooled_vector = np.mean(vectors_array, axis=0)
    return mean_pooled_vector

In [ ]:
def embed_product_details(product):
    name = product['Name']
    dict_string = get_product_details(product['Specifications'])
    colour = dict_string['Color']
    pattern = dict_string['Pattern']
    fabric = dict_string['Fabric']
    fit = dict_string['Fit']
    type = dict_string['Type']
    embedding = getTextEmbeddings(f"Name: {name}, Color: {colour}, Pattern: {pattern}, Fabric: {fabric}, Fit: {fit}, Type: {type}")
    return embedding.tolist()[0]

In [ ]:
list_of_products = [0, 3, 4, 5, 8, 9]

In [ ]:
for i in list_of_products:
    user_profile4_bottom.append(embed_product_details(df5.iloc[i]))

In [ ]:
mean_pooling(user_profile1_top).tolist()

In [12]:
client.data_object.create(
    {
        "name":"Ginny Weasley",
        "gender":"Female",
        "age":"32",
        "category":"Bottom"
    },
    class_name="Customer_Profile",
    uuid=generate_uuid5("Female Profile 2 Bottom"),
    vector=getTextEmbeddings("Jeans, tight and loose fit but jeans")
)

'8d722619-721c-5bcc-a866-fc53263f58b7'